# Create ENV

In [9]:
from azureml.core.runconfig import CondaDependencies

cd = CondaDependencies.create()
cd.add_conda_package("numpy")
cd.add_conda_package('python=3.8')
cd.add_conda_package('tensorflow')
cd.add_conda_package('pip')

cd.add_pip_package("PyWavelets")
cd.add_pip_package("matplotlib")
cd.add_pip_package("azureml-defaults")


cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')


'myenv.yml'

# Deploy ACI

In [2]:
import urllib.request
from azureml.core.model import Model
from azureml.core import Workspace

ws = Workspace(subscription_id="d1dd79d8-6a20-4de8-bc99-29e507321f02",
               resource_group="GR-CRRS-DATA",
               workspace_name="ml-data-studio")

In [11]:
from azureml.core.model import Model

model = Model(ws,'heartCNN_VGG16_DV5')

In [12]:
model

Model(workspace=Workspace.create(name='ml-data-studio', subscription_id='d1dd79d8-6a20-4de8-bc99-29e507321f02', resource_group='GR-CRRS-DATA'), name=heartCNN_VGG16_DV5, id=heartCNN_VGG16_DV5:1, version=1, tags={}, properties={})

In [13]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core import Environment


myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="echo_score.py", environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=3, 
                                               memory_gb=6, 
                                               tags={'Red':'VGG16', 'framework': 'TensorFlow'},
                                               description='Tensorflow Medical Image Classification CNN and CWT')

service = Model.deploy(workspace=ws, 
                           name='ia-classification-medical-vision', 
                           models=[model], 
                           inference_config= inference_config, 
                           deployment_config= aciconfig,
                           overwrite=True)

service.wait_for_deployment(True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-04-30 15:48:51+00:00 Creating Container Registry if not exists.
2022-04-30 15:48:51+00:00 Registering the environment.
2022-04-30 15:48:52+00:00 Building image..
2022-04-30 15:56:21+00:00 Generating deployment configuration.
2022-04-30 15:56:23+00:00 Submitting deployment to compute.
2022-04-30 15:56:25+00:00 Checking the status of deployment ia-classification-medical-vision..
2022-04-30 16:01:27+00:00 Checking the status of inference endpoint ia-classification-medical-vision.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [15]:
print(service.get_logs())

2022-04-30T16:00:27,700665500+00:00 - gunicorn/run 
2022-04-30T16:00:27,700951400+00:00 - iot-server/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-04-30T16:00:27,704293000+00:00 - rsyslog/run 
2022-04-30T16:00:27,720978300+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-04-30T16:00:27,932539800+00:00 - iot-server/finish 1 0
2022-04-30T16:00:27,934876200+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (86)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 113
SPARK_HOME not set. Skipping PySpark Initialization.
generated new fontManager
Initializing logger
2022-04-30 16:00:32,655 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2022-04-30 16:00:32,656 | root | INFO | Starting up request id generator
2022-04-30 16:00:

In [3]:
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core import Environment



In [9]:
myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="echo_score.py", environment=myenv)

service_name = 'ia-classification-medical-vision'
# Retrieve existing service.
service = AciWebservice(name=service_name, workspace=ws)

In [10]:
service

AciWebservice(workspace=Workspace.create(name='ml-data-studio', subscription_id='d1dd79d8-6a20-4de8-bc99-29e507321f02', resource_group='GR-CRRS-DATA'), name=ia-classification-medical-vision, image_id=None, image_digest=None, compute_type=ACI, state=Healthy, scoring_uri=http://8d516c58-5deb-420a-b634-adbe3d3fa449.southcentralus.azurecontainer.io/score, tags={'Red': 'VGG16', 'framework': 'TensorFlow'}, properties={'hasInferenceSchema': 'False', 'hasHttps': 'False'}, created_by={'userObjectId': '3aa7f6df-9a9e-4d46-80fb-3b28b6d99e42', 'userPuId': '100300009B544CC4', 'userIdp': None, 'userAltSecId': None, 'userIss': 'https://sts.windows.net/0e0cb060-09ad-49f5-a005-68b9b49aa1f6/', 'userTenantId': '0e0cb060-09ad-49f5-a005-68b9b49aa1f6', 'userName': 'Christian Ruben Ramos Sanchez', 'upn': 'u201710131@upc.edu.pe'})

In [18]:
service.update(inference_config=inference_config)
service.wait_for_deployment(show_output=True)
print(service.state)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-04-30 23:05:35+00:00 Creating Container Registry if not exists.
2022-04-30 23:05:35+00:00 Registering the environment.
2022-04-30 23:05:36+00:00 Use the existing image.
2022-04-30 23:05:36+00:00 Generating deployment configuration.
2022-04-30 23:05:37+00:00 Submitting deployment to compute.
2022-04-30 23:05:39+00:00 Checking the status of deployment ia-classification-medical-vision..
2022-04-30 23:09:52+00:00 Checking the status of inference endpoint ia-classification-medical-vision.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [12]:
print(service.get_logs())

2022-04-30T22:03:39,077225500+00:00 - gunicorn/run 
2022-04-30T22:03:39,077788600+00:00 - iot-server/run 
2022-04-30T22:03:39,079957900+00:00 - rsyslog/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-04-30T22:03:39,088256400+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-04-30T22:03:39,287908200+00:00 - iot-server/finish 1 0
2022-04-30T22:03:39,290541000+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (85)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 114
SPARK_HOME not set. Skipping PySpark Initialization.
generated new fontManager
Initializing logger
2022-04-30 22:03:43,850 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2022-04-30 22:03:43,851 | root | INFO | Starting up request id generator
2022-04-30 22:03: